# Well data processing

In [86]:
from pathlib import Path
import os
import sys
import numpy as np
from matplotlib import pyplot as plt
import lasio
import pandas as pd
import seaborn as sns
import matplotlib.cm as cm
import glob
from scipy.stats import mode
import warnings
sns.set(color_codes=True)

In [100]:
workdir = Path('../')
lasdir = workdir.joinpath('data','las_MPWSP')
# lasdir = workdir.joinpath('data','las_all')

auxdir=  workdir.joinpath('data','auxiliary')
figdir=  workdir.joinpath('work','figs')

In [173]:
def myround(x, base=.5):
    return base*np.round(x/base)
def mask_nonan(df,nan_col_list):
    mask = df.notnull()
    return mask.loc[:,nan_col_list].all(axis=1)

def mask_noclay(df,nan_col_list):
    return ~df.loc[mask_nonan(df,nan_col_list),'lith'].str.contains("C")

def mask_SP_only(df,nan_col_list):
    return df.loc[mask_nonan(df,nan_col_list),'lith']=="SP"

def mask_stds(df,nstds,testcol,nan_col_list):
    pop = df.loc[mask_nonan(df,nan_col_list),testcol]
    return  np.logical_and(pop < np.mean(pop)+nstds*np.std(pop),
                           pop > np.mean(pop)-nstds*np.std(pop))
def mask_wherescreen(df,nan_col_list):
    return  mask_nonan(df[df.screen.notnull()],nan_cols)

def tds2rho_est(tds,m=.692826,b=-220.28):
    return 1e4*m/(tds-b)

def rho2tds_est(rho,m=.692826,b=-220.28):
    return 1e4*m/rho + b

def cond2rho(cond):
    return 1e4/cond
    
def nested_logicals(df,list_of_conditions,func=np.logical_and):
    '''
    Convenience function to nest numpy logical functions
        Input: 
            df
            list_of_conditions
            func: numpy function to be applied
        Output:
            logical mask for the given dataframe
        '''
    out = np.ones(len(list_of_conditions[0]),dtype=bool)
    for i in range(len(list_of_conditions)):
        out = func(out,list_of_conditions[i])
    return out

def interpIntervals(df,interp_cols,group_cols=['Well','Year'],dept_col='DEPT',interval=.5):
    '''
    Interpolates numerical values to a specified resolution (interval) 
    Meant for use with depth-registered well data
        Input:
            df: DataFrame
            interp_cols: list
            group_cols: list
            dept_col: str
            interval: numeric
        Output:
            DataFrame with interpolated rows
    '''
    frames = []
    for key,group in df.groupby(by=group_cols):

        all_cols = group.columns
        dup_cols=[col for col in all_cols if (col not in (interp_cols+[dept_col]))]

        #interpolation
        x_eval = np.arange(group.loc[:,dept_col].values[0],group.loc[:,dept_col].values[-1]+interval,interval)
        xp = group.loc[:,dept_col].values
        y_eval={}
        for col in interp_cols:
            yp = group.loc[:,col].values
            y_eval[col]=np.interp(x_eval,xp,yp)

        #create new DF
        df_out = pd.DataFrame(columns=all_cols)
        df_out = df_out.append([df.loc[group.index[0],dup_cols]]*len(x_eval))
        for col in interp_cols:
            df_out.loc[:,col] = y_eval[col]
        df_out.loc[:,dept_col] = x_eval

        #concatenate frames
        frames.append(df_out)
    return pd.concat(frames).reset_index(drop=True)

def concat_dfs(dfs):
    dfs = [sc_df_interp,df2]
    new_dfs = []
    maxind= 0

    for df in dfs:
        df_index = df.index.values + maxind + 1
        maxind = np.max(df_index)
        df.loc[:,'new_index'] = df_index
        df.set_index('new_index',inplace=True)
        new_dfs.append(df)
    return pd.concat(new_dfs,axis=0,sort=True)

def update_df2():
    return df.reset_index().rename(columns={"level_0": "Well"})

def update_FBS(df,TDS_col):
    FBS = np.zeros(len(df))
    FBS[df.loc[:,TDS_col].isna()] = np.nan
    FBS[df.loc[:,TDS_col] <= 3000] = 1
    FBS[np.logical_and(df.loc[:,TDS_col] > 3000,df.loc[:,TDS_col] < 10000)] = 2
    FBS[df.loc[:,TDS_col] >= 10000] = 3
    return df.assign(FBS=FBS)

def m2ft(df,colname,newcolname=None):
    if newcolname is None:
        newcolname = colname + 'ft'
    df[newcolname] = df[colname]/.3048
    return

def interval2point(pointdf,intervaldf,testcol,topcol='topft',botcol='botft'):
    df_list = []
    for d in pointdf.index:
        v=intervaldf[((d>=intervaldf[topcol]) & (d<intervaldf[botcol]))][testcol]
        if len(v)==1:
            df_list.append(v.values[0])
        elif len(v)>1:
            print(v)
            raise Exception('multiple values in this interval ??')
        else:
            df_list.append(np.nan)
    return df_list


In [206]:
recompute = False
clip_bad_data=True

if not recompute:
    df =  pd.read_pickle(lasdir.joinpath('allMWs'))
    df_ind =  pd.read_pickle(lasdir.joinpath('allMWs_ind'))
    MWnames = df.index.levels[0].values
else:
    f_MWm = glob.glob(lasdir.joinpath('*[1-9]m.las').as_posix())
    for f in f_MWm:
        lasobj = lasio.read(f)
        lasexpt = lasio.LASFile()

        for c in lasobj.curves:
            if c.mnemonic=='DEPT':
                continue
            lasexpt.insert_curve_item(0,c)
        #make sure DEPT is the first curve for aesthetics
        d = lasobj.get_curve('DEPT')
        d_mode = mode(np.diff(np.round(d.data/.3048,decimals=2))).mode[0]
        if d_mode==0.5:
            lasexpt.insert_curve(0,'DEPT',myround(d.data/.3048,base=.5),unit='F')
            lasexpt.write(f[:-5]+'_all.las')
        else:
            raise Exception('Needs to be .5 feet')

    ########## Create list of DataFrames
    f_MWall = glob.glob(lasdir.joinpath('*_all.las').as_posix())
    f_MWall = sorted(f_MWall,key=os.path.basename)
    MWnames = [Path(f).stem[:-4] for f in f_MWall]

    frames = []
    for f in f_MWall:
        las = lasio.read(f)
        #Convert to DF and mask for plots
        df_temp = las.df()
        frames.append(df_temp)

    ######### Meters to feet for lith and screen
    def m2ft(df,colname,newcolname=None):
        if newcolname is None:
            newcolname = colname + 'ft'
        df[newcolname] = df[colname]/.3048
        return

    mwcollar = pd.read_excel(auxdir.joinpath('MPWSP_Collar.xlsx'))
    mwlith= pd.read_excel(auxdir.joinpath('Lith_MPWSP.xlsx'))
    mwtds= pd.read_excel(auxdir.joinpath('TDS_Baseline_MPWSP.xlsx'),index_col=0)
    mwscreen = pd.read_excel(auxdir.joinpath('MPWSP_Screen.xlsx'),index_col=0)
    mwchem = pd.read_excel(auxdir.joinpath('WQ_Baseline_MPWSP.xlsx'),index_col=0)
    wellids = np.unique(mwlith['WellID'])


    m2ft(mwlith,'Top_m','topft')
    m2ft(mwlith,'Bottom_m','botft')
    m2ft(mwscreen,'Screen_Top','topft')
    m2ft(mwscreen,'Screen_Bottom','botft')
    
    for i,frame in enumerate(frames):
        nam = MWnames[i]
        print('processing',nam)
        TDS1_list = np.nan*np.zeros(len(frame))
        TDS2_list = np.nan*np.zeros(len(frame))
        EC1_list = np.nan*np.zeros(len(frame))
        EC2_list = np.nan*np.zeros(len(frame))
        EC1f_list = np.nan*np.zeros(len(frame))
        EC2f_list = np.nan*np.zeros(len(frame))
        TMPf1_list = np.nan*np.zeros(len(frame))
        TMPf2_list = np.nan*np.zeros(len(frame))

        #Assign X and Y
        frame = frame.assign(X=mwcollar.loc[mwcollar.Well==nam,'X'].values[0]*np.ones(len(frame)))
        frame = frame.assign(Y=mwcollar.loc[mwcollar.Well==nam,'Y'].values[0]*np.ones(len(frame)))
        frame = frame.assign(Region=list(mwcollar.loc[mwcollar.Well==nam,'Region'].values)*len(frame))

        #Assign lith
        lith_list = interval2point(frame,mwlith.loc[lambda df: df['WellID'] == nam]
                   ,'Lith_Code_Orig',topcol='topft',botcol='botft')
        frame = frame.assign(lith=lith_list)

        #Assign screen and aquifer name
        screen_list = interval2point(frame,mwscreen.loc[lambda df: df.index == nam]
                   ,'Alt_ID',topcol='topft',botcol='botft')
        screen_len_list = interval2point(frame,mwscreen.loc[lambda df: df.index == nam]
                   ,'Screen_Len_ft',topcol='topft',botcol='botft')
        aq_list = interval2point(frame,mwscreen.loc[lambda df: df.index == nam]
                   ,'TargetedAquifer',topcol='topft',botcol='botft')
        aq_group_list = interval2point(frame,mwscreen.loc[lambda df: df.index == nam]
                   ,'GroupedAquifer',topcol='topft',botcol='botft')

        frame = frame.assign(screen=screen_list)
        frame = frame.assign(screen_len=screen_len_list)
        frame = frame.assign(Aquifer=aq_list)
        frame = frame.assign(AquiferGroup=aq_group_list)

        #Assign TDS measurements to each screen
        for scrn in frame.screen.dropna().unique():
            framebool = frame['screen']==scrn #entries in frame matching the well and screen
            screenbool = np.logical_and(mwtds.index == nam,mwtds['WellScreen'] == scrn) #matching entries in mwtds
            chembool = np.logical_and(mwchem.index == nam,mwchem['WellScreen'] == scrn)
            nmeas = len(mwtds.loc[screenbool,'TDS(mg/L)'])
            if nmeas==2:
                TDS1_list[framebool] =mwtds.loc[screenbool,'TDS(mg/L)'].iloc[0]
                TDS2_list[framebool] =mwtds.loc[screenbool,'TDS(mg/L)'].iloc[1]     
                EC1_list[framebool] =mwchem.loc[chembool,'SPECIFIC CONDUCTANCE (E.C)'].iloc[0]
                EC2_list[framebool] =mwchem.loc[chembool,'SPECIFIC CONDUCTANCE (E.C)'].iloc[1] 
                EC1f_list[framebool] =mwchem.loc[chembool,'SPECIFIC CONDUCTANCE (E.C) (FIELD)'].iloc[0]
                EC2f_list[framebool] =mwchem.loc[chembool,'SPECIFIC CONDUCTANCE (E.C) (FIELD)'].iloc[1] 
                TMPf1_list[framebool] =mwchem.loc[chembool,'TEMPERATURE, (FIELD)'].iloc[0]
                TMPf2_list[framebool] =mwchem.loc[chembool,'TEMPERATURE, (FIELD)'].iloc[1] 
            elif nmeas==1:
                TDS1_list[framebool] =mwtds.loc[screenbool,'TDS(mg/L)'].iloc[0]
                try:
                    TMPf1_list[framebool] =mwchem.loc[chembool,'TEMPERATURE, (FIELD)'].iloc[0]
                    EC1_list[framebool] =mwchem.loc[chembool,'SPECIFIC CONDUCTANCE (E.C)'].iloc[0]
                    EC1f_list[framebool] =mwchem.loc[chembool,'SPECIFIC CONDUCTANCE (E.C) (FIELD)'].iloc[0]
                except:
                    print("No chem data found for well {} screen {}".format(nam,scrn))
            else:
                print('Num of measurements:',nmeas)
                raise Exception('Number of measurements not equal to 1 or 2')

        frame = frame.assign(TDS1=TDS1_list)
        frame = frame.assign(TDS2=TDS2_list)
        frame = frame.assign(EC1=EC1_list)
        frame = frame.assign(EC2=EC2_list)
        frame = frame.assign(EC1f=EC1f_list)
        frame = frame.assign(EC2f=EC2f_list)
        frame = frame.assign(TMPf1=TMPf1_list)
        frame = frame.assign(TMPf2=TMPf2_list)
        frame[frame.loc[:,['TDS1','TDS2','EC1','EC2','EC1f','EC2f','TMPf1','TMPf2']]==0]=np.nan

        #Assign to list
        frames[i] = frame

    ######### Join all frames together
    df = pd.concat(frames, keys=MWnames, sort=False)


    ######### Replace 'Gravelly/silty sand w/clay' with 'SC-SM'
    df.loc[df.lith == 'Gravelly/silty sand w/clay','lith'] = 'SC-SM'

    ######### Add FRESc
    def degc2f(degc):
        return (degc*9/5)+32
    ref_temp= 77 #fahrenheit
    fresc = df.loc[:,'FRES']*(df.loc[:,'TMP']+6.77)/(ref_temp + 6.77)
    df=df.assign(FRESc = fresc);
    ec_fresc = cond2rho(df.loc[:,'EC1f'])*(degc2f(df.loc[:,'TMPf1'])+6.77)/(ref_temp + 6.77)
    df=df.assign(ec_fresc=ec_fresc)

    ######### Add aquifer number
    aqnum = -1*np.ones(len(df),dtype='Int32')
    aq_names = df[df.Aquifer.notna()].Aquifer.unique()
    aq_names.sort()
    for i,aq in enumerate(aq_names):
        aqnum[df.Aquifer == aq_names[i]] = i

    df=df.assign(aqnum=aqnum)

    
    if clip_bad_data:
        ######### Record intervals of bad data at top and bottom of borehole
        topft = np.zeros(np.shape(MWnames))
        botft = np.zeros(np.shape(MWnames))
        data_rng = pd.DataFrame(np.r_[[MWnames,topft,botft]].T,columns=['Well','topft','botft'])
        def assign_top_bot(df,nam,top,bot):
            df.loc[df['Well']==nam,['topft','botft']] = (top,bot)
        assign_top_bot(data_rng,'MW-1D',100 ,330)
        assign_top_bot(data_rng,'MW-4D',0   ,331.0)
        assign_top_bot(data_rng,'MW-5D',75   ,426.5)
        assign_top_bot(data_rng,'MW-6D',40   ,400)
        assign_top_bot(data_rng,'MW-7D',70  ,345)
        assign_top_bot(data_rng,'MW-8D',30   ,350)
        assign_top_bot(data_rng,'MW-9D',5   ,392)
#         #Do some manual clipping of bad data at the top and bottom of tracks
#         top,bot =data_rng.loc[data_rng['Well']==nam,['topft','botft']].values[0]
#         frame.loc[frame.index<top,['FRES','RILD']] = np.nan
#         frame.loc[frame.index>bot,['FRES','RILD']] = np.nan   

    
    ######### Save DF
    df_ind = pd.DataFrame(df.index.tolist(), columns=['Well','DEPT'])

    df.to_pickle(lasdir.joinpath('allMWs'))
    df_ind.to_pickle(lasdir.joinpath('allMWs_ind'))
    print('Done reprocessing from raw data!')

In [207]:
df.Aquifer.unique()

array([nan, 'Dune_Sand', '180-FTE', '180/400-Foot Aquitard', '400-Foot',
       'A_Perched', '180-Foot', 'A', '180-Foot_Lower'], dtype=object)

### Get Soquel Creek data and interpolate to .5 feet

In [208]:
sc_df = pd.read_excel(auxdir.joinpath('SoquelCreek.xlsx'))
sc_df_interp = interpIntervals(sc_df,interp_cols = ['RILD'],group_cols=['Well','Year'],dept_col='DEPT',interval=.5)
sc_df_interp.to_pickle(lasdir.joinpath('SoquelCreek'))


df2 = update_df2()
df2.to_pickle(lasdir.joinpath('allMWs_df2'))

# df = concat_dfs((df2,sc_df_interp.set_index(['Well','DEPT'])))
df = pd.concat((df,sc_df_interp.set_index(['Well','DEPT'])),sort=False)

#Add Rw_est
Rw_est = np.zeros(len(df))
Rw_est[~df.TDS1.isna()] = tds2rho_est(df[~df.TDS1.isna()].TDS1)
Rw_est[~df.EC1f.isna()]=cond2rho(df[~df.EC1f.isna()].EC1f)
Rw_est[df.TDS1.isna()]=np.nan
df = df.assign(Rw_est=Rw_est)

df_ind = pd.DataFrame(df.index.tolist(), columns=['Well','DEPT'])
df.to_pickle(lasdir.joinpath('allMWs_SC'))
df_ind.to_pickle(lasdir.joinpath('allMWs_SC_ind'))
print('Done reprocessing from raw data!')

/Users/ianpg/anaconda/envs/SWIenv/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Done reprocessing from raw data!
